In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
census_df = pd.read_csv('co-est2019-alldata.csv', encoding='latin-1')
census_df = census_df[census_df.SUMLEV == 50]
census_df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57376,...,1.020788,0.208812,-1.650165,-0.347225,-2.049590,-1.338525,-1.391062,6.193562,-0.069229,1.124597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43574,...,-17.605427,0.502513,15.035735,-4.614071,-13.140225,-13.762031,-18.484164,-23.515727,-18.743773,-17.065527
3189,50,4,8,56,39,Wyoming,Teton County,21294,21298,21296,...,-1.583464,-2.575509,1.905293,23.259519,12.508872,4.670450,1.253214,-0.171608,-12.432212,1.797445
3190,50,4,8,56,41,Wyoming,Uinta County,21118,21121,21089,...,-9.181105,-18.196975,-4.487730,-10.870861,-15.033634,-10.484550,-11.485101,-18.923455,-13.554993,-9.181105
3191,50,4,8,56,43,Wyoming,Washakie County,8533,8528,8530,...,-7.652085,-12.839390,-3.084589,-1.307811,-18.938032,0.000000,-15.204038,-15.950541,-17.246806,-8.289759


In [3]:
combine_fips = lambda a, b: '{:0>2}{:0>3}'.format(a, b)
census_df['FIPS'] = census_df.STATE.combine(census_df.COUNTY, combine_fips).astype(int)
census_df = census_df[['FIPS', 'POPESTIMATE2019']].set_index('FIPS')

fip_pop_dict = census_df.POPESTIMATE2019.to_dict()
census_df

,POPESTIMATE2019
FIPS,
1001,55869
1003,223234
1005,24686
1007,22394
1009,57826
...,...
56037,42343
56039,23464
56041,20226


In [4]:
# JHU Dataset 
# https://github.com/CSSEGISandData/COVID-19
# set dates based files in COVID-19\csse_covid_19_data\csse_covid_19_daily_reports_us
dates = [f'04-{12+i:02d}-2020' for i in range(19)]
dates += [f'05-{1+i:02d}-2020' for i in range(4)]

def get_jhu_table(date):
    df = pd.read_csv(f'git/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/{date}.csv')
    df['date'] = pd.to_datetime(date)
    
    df.dropna(subset=['FIPS'], inplace=True)
    df.FIPS = df.FIPS.astype(int)
    df = df[df.FIPS.isin(census_df.index)]
    
    return df
    
jhu_corona_df = pd.concat([get_jhu_table(i) for i in dates])
jhu_corona_df

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,date
0,45001,Abbeville,South Carolina,US,2020-04-12 23:18:00,34.223334,-82.461707,9,0,0,9,"Abbeville, South Carolina, US",2020-04-12
1,22001,Acadia,Louisiana,US,2020-04-12 23:18:00,30.295065,-92.414197,99,5,0,94,"Acadia, Louisiana, US",2020-04-12
2,51001,Accomack,Virginia,US,2020-04-12 23:18:00,37.767072,-75.632346,15,0,0,15,"Accomack, Virginia, US",2020-04-12
3,16001,Ada,Idaho,US,2020-04-12 23:18:00,43.452658,-116.241552,517,6,0,511,"Ada, Idaho, US",2020-04-12
4,19001,Adair,Iowa,US,2020-04-12 23:18:00,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US",2020-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924,2290,Yukon-Koyukuk,Alaska,US,2020-05-05 02:32:34,65.508155,-151.390739,1,0,0,1,"Yukon-Koyukuk, Alaska, US",2020-05-04
2925,4027,Yuma,Arizona,US,2020-05-05 02:32:34,32.768957,-113.906667,115,1,0,114,"Yuma, Arizona, US",2020-05-04
2926,8125,Yuma,Colorado,US,2020-05-05 02:32:34,40.003468,-102.425867,9,0,0,9,"Yuma, Colorado, US",2020-05-04
2927,48505,Zapata,Texas,US,2020-05-05 02:32:34,27.001564,-99.169872,7,0,0,7,"Zapata, Texas, US",2020-05-04


In [5]:
jhu_pivot = jhu_corona_df.pivot('FIPS', 'date').Confirmed
jhu_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,...,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,24.0,26.0,26.0,25.0,26.0,28.0,30.0,...,36.0,37.0,39.0,40.0,43.0,44.0,42.0,45.0,48.0,53.0
1003,71.0,72.0,87.0,91.0,101.0,103.0,109.0,112.0,117.0,123.0,...,147.0,161.0,168.0,171.0,174.0,174.0,175.0,181.0,187.0,188.0
1005,10.0,10.0,11.0,12.0,14.0,15.0,18.0,20.0,22.0,28.0,...,32.0,33.0,35.0,37.0,37.0,39.0,42.0,43.0,45.0,45.0
1007,16.0,17.0,17.0,18.0,22.0,24.0,26.0,28.0,32.0,32.0,...,34.0,38.0,42.0,42.0,42.0,42.0,42.0,42.0,43.0,42.0
1009,13.0,14.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,...,31.0,34.0,34.0,34.0,36.0,37.0,39.0,40.0,40.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,16.0,...,16.0,16.0,16.0,16.0,17.0,18.0,18.0,18.0,18.0,18.0
56039,56.0,56.0,57.0,58.0,59.0,61.0,62.0,62.0,62.0,92.0,...,95.0,95.0,95.0,96.0,96.0,96.0,96.0,97.0,98.0,98.0


In [6]:
# NYTimes Data
# https://github.com/nytimes/covid-19-data
start_date_nyt = '2020-04-12' # Match the John Hopkins Data

nyt_corona_df = pd.read_csv('git/covid-19-data/us-counties.csv')
nyt_corona_df.date = pd.to_datetime(nyt_corona_df.date)

nyt_corona_df.rename(columns={'fips': 'FIPS'}, inplace=True)
nyt_corona_df.dropna(subset=['FIPS'], inplace=True)
nyt_corona_df.FIPS = nyt_corona_df.FIPS.astype(int)
nyt_corona_df = nyt_corona_df[nyt_corona_df.FIPS.isin(census_df.index)]

nyt_corona_df = nyt_corona_df[nyt_corona_df.date >= start_date_nyt]
nyt_corona_df

,date,county,state,FIPS,cases,deaths
51165,2020-04-12,Autauga,Alabama,1001,19,1
51166,2020-04-12,Baldwin,Alabama,1003,71,1
51167,2020-04-12,Barbour,Alabama,1005,10,0
51168,2020-04-12,Bibb,Alabama,1007,16,0
51169,2020-04-12,Blount,Alabama,1009,13,0
...,...,...,...,...,...,...
115452,2020-05-04,Sublette,Wyoming,56035,1,0
115453,2020-05-04,Sweetwater,Wyoming,56037,11,0
115454,2020-05-04,Teton,Wyoming,56039,67,1
115455,2020-05-04,Uinta,Wyoming,56041,6,0


In [7]:
nyt_pivot = nyt_corona_df.pivot('FIPS', 'date').cases
nyt_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,...,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,25.0,25.0,26.0,25.0,27.0,28.0,30.0,...,37.0,37.0,39.0,40.0,43.0,42.0,42.0,45.0,48.0,53.0
1003,71.0,78.0,87.0,98.0,102.0,103.0,109.0,114.0,117.0,123.0,...,154.0,161.0,168.0,171.0,173.0,174.0,175.0,181.0,187.0,188.0
1005,10.0,10.0,11.0,13.0,14.0,15.0,18.0,20.0,22.0,28.0,...,33.0,33.0,35.0,37.0,37.0,39.0,42.0,43.0,45.0,45.0
1007,16.0,17.0,17.0,19.0,24.0,23.0,26.0,28.0,32.0,32.0,...,35.0,38.0,42.0,42.0,42.0,42.0,42.0,42.0,43.0,42.0
1009,13.0,15.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,...,31.0,34.0,34.0,34.0,36.0,37.0,39.0,40.0,40.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,11.0,11.0,11.0,11.0,11.0
56039,56.0,56.0,57.0,59.0,59.0,61.0,62.0,62.0,62.0,62.0,...,64.0,64.0,64.0,65.0,64.0,65.0,65.0,66.0,67.0,67.0


In [8]:
#combine John Hopkins and NYT data, prioritizing John Hopkins
take_first = lambda s1, s2: s1 if s1 is not None else s2
corona_pivot = jhu_pivot.combine(nyt_pivot, take_first)
corona_pivot = corona_pivot.bfill(axis = 1).ffill(axis = 1).dropna()
corona_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,...,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,24.0,26.0,26.0,25.0,26.0,28.0,30.0,...,36.0,37.0,39.0,40.0,43.0,44.0,42.0,45.0,48.0,53.0
1003,71.0,72.0,87.0,91.0,101.0,103.0,109.0,112.0,117.0,123.0,...,147.0,161.0,168.0,171.0,174.0,174.0,175.0,181.0,187.0,188.0
1005,10.0,10.0,11.0,12.0,14.0,15.0,18.0,20.0,22.0,28.0,...,32.0,33.0,35.0,37.0,37.0,39.0,42.0,43.0,45.0,45.0
1007,16.0,17.0,17.0,18.0,22.0,24.0,26.0,28.0,32.0,32.0,...,34.0,38.0,42.0,42.0,42.0,42.0,42.0,42.0,43.0,42.0
1009,13.0,14.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,...,31.0,34.0,34.0,34.0,36.0,37.0,39.0,40.0,40.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,16.0,...,16.0,16.0,16.0,16.0,17.0,18.0,18.0,18.0,18.0,18.0
56039,56.0,56.0,57.0,58.0,59.0,61.0,62.0,62.0,62.0,92.0,...,95.0,95.0,95.0,96.0,96.0,96.0,96.0,97.0,98.0,98.0


In [9]:
# set how many days it takes for an average case to resolve
num_days = 14

In [10]:
sir_df = pd.DataFrame(corona_pivot.iloc[:, -1].rename("cases_now"))
sir_df['cases_old'] = corona_pivot.iloc[:, -num_days]
display(sir_df.describe())
sir_df

,cases_now,cases_old
count,2875.000000,2875.000000
mean,406.720348,278.641043
std,3776.457920,3003.260031
min,0.000000,0.000000
25%,6.000000,4.000000
50%,25.000000,15.000000
75%,108.000000,65.000000
max,175651.000000,144190.000000


,cases_now,cases_old
FIPS,,
1001,53.0,30.0
1003,188.0,123.0
1005,45.0,28.0
1007,42.0,32.0
1009,40.0,26.0
...,...,...
56035,3.0,3.0
56037,18.0,16.0
56039,98.0,92.0


In [11]:
sir_df['POPULATION'] = [fip_pop_dict[i] for i in sir_df.index]
sir_df['SUSCEPTIBLE'] = (sir_df.POPULATION - sir_df.cases_now)/sir_df.POPULATION
sir_df['INFECTED'] = (sir_df.cases_now - sir_df.cases_old)/sir_df.POPULATION
sir_df['REMOVED'] = (sir_df.cases_old)/sir_df.POPULATION
sir_df.SUSCEPTIBLE = [float('NaN') if i>1 or i<0 else i for i in sir_df.SUSCEPTIBLE]
sir_df.INFECTED = [float('NaN') if i>1 or i<0 else i for i in sir_df.INFECTED]
sir_df.REMOVED = [float('NaN') if i>1 or i<0 else i for i in sir_df.REMOVED]
sir_df = sir_df[['SUSCEPTIBLE', 'INFECTED', 'REMOVED']].dropna()
sir_df

,SUSCEPTIBLE,INFECTED,REMOVED
FIPS,,,
1001,0.999051,0.000412,0.000537
1003,0.999158,0.000291,0.000551
1005,0.998177,0.000689,0.001134
1007,0.998124,0.000447,0.001429
1009,0.999308,0.000242,0.000450
...,...,...,...
56035,0.999695,0.000000,0.000305
56037,0.999575,0.000047,0.000378
56039,0.995823,0.000256,0.003921


In [12]:
# calculate number of people infected everyday
corona_pivot_diff = corona_pivot.diff(axis=1)
corona_pivot_diff.dropna(axis=1, how='all', inplace=True)
corona_pivot_diff

date,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,...,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,0.0,4.0,1.0,2.0,0.0,-1.0,1.0,2.0,2.0,2.0,...,0.0,1.0,2.0,1.0,3.0,1.0,-2.0,3.0,3.0,5.0
1003,1.0,15.0,4.0,10.0,2.0,6.0,3.0,5.0,6.0,9.0,...,0.0,14.0,7.0,3.0,3.0,0.0,1.0,6.0,6.0,1.0
1005,0.0,1.0,1.0,2.0,1.0,3.0,2.0,2.0,6.0,1.0,...,0.0,1.0,2.0,2.0,0.0,2.0,3.0,1.0,2.0,0.0
1007,1.0,0.0,1.0,4.0,2.0,2.0,2.0,4.0,0.0,2.0,...,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
1009,1.0,2.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,3.0,...,0.0,3.0,0.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56037,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
56039,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,30.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [13]:
# normalize by dividing by population
corona_pivot_diff_pct = corona_pivot_diff.apply(lambda x: x/[fip_pop_dict[i] for i in x.index])
corona_pivot_diff_pct

date,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,...,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,0.000000,0.000072,0.000018,0.000036,0.000000,-0.000018,0.000018,0.000036,0.000036,0.000036,...,0.0,0.000018,0.000036,0.000018,0.000054,0.000018,-0.000036,0.000054,0.000054,0.000089
1003,0.000004,0.000067,0.000018,0.000045,0.000009,0.000027,0.000013,0.000022,0.000027,0.000040,...,0.0,0.000063,0.000031,0.000013,0.000013,0.000000,0.000004,0.000027,0.000027,0.000004
1005,0.000000,0.000041,0.000041,0.000081,0.000041,0.000122,0.000081,0.000081,0.000243,0.000041,...,0.0,0.000041,0.000081,0.000081,0.000000,0.000081,0.000122,0.000041,0.000081,0.000000
1007,0.000045,0.000000,0.000045,0.000179,0.000089,0.000089,0.000089,0.000179,0.000000,0.000089,...,0.0,0.000179,0.000179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000045,-0.000045
1009,0.000017,0.000035,0.000017,0.000017,0.000035,0.000000,0.000017,0.000017,0.000069,0.000052,...,0.0,0.000052,0.000000,0.000000,0.000035,0.000017,0.000035,0.000017,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000203,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56037,0.000047,0.000000,0.000024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000142,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000024,0.000024,0.000000,0.000000,0.000000,0.000000
56039,0.000000,0.000043,0.000043,0.000043,0.000085,0.000043,0.000000,0.000000,0.001279,0.000043,...,0.0,0.000000,0.000000,0.000043,0.000000,0.000000,0.000000,0.000043,0.000043,0.000000


In [14]:
# Calculate average change 

def func(x):
    change_pcts = [i for i in x if i != 0]
    avg = np.sum(change_pcts)/len(x)
    
    # drop samples without too few changes or negative change
    if len(change_pcts) < 3 or avg <= 0:
        return float("NaN")
    
    return avg

average_row = lambda x: func(x)

hyperparam_df = pd.DataFrame(corona_pivot_diff_pct.iloc[:, -num_days:].T.apply(average_row).rename("beta"))
hyperparam_df['gamma'] = corona_pivot_diff_pct.iloc[:, :-num_days].T.apply(average_row)
corona_quartiles = pd.concat([sir_df.describe(), hyperparam_df.describe()], axis=1)
display(hyperparam_df)
corona_quartiles

,beta,gamma
FIPS,,
1001,0.000032,0.000020
1003,0.000023,0.000026
1005,0.000067,0.000061
1007,0.000032,0.000089
1009,0.000022,0.000019
...,...,...
56035,NaN,NaN
56037,0.000013,NaN
56039,0.000110,0.000032


,SUSCEPTIBLE,INFECTED,REMOVED,beta,gamma
count,2852.000000,2852.000000,2852.000000,1.974000e+03,1.562000e+03
mean,0.998079,0.000808,0.001113,8.560765e-05,7.634731e-05
std,0.004712,0.003155,0.002724,2.716570e-04,1.822378e-04
min,0.880539,0.000000,0.000000,6.436166e-07,1.694066e-21
25%,0.998170,0.000065,0.000236,1.453989e-05,1.717532e-05
50%,0.999236,0.000225,0.000489,3.274113e-05,3.451866e-05
75%,0.999629,0.000719,0.001077,7.933236e-05,7.694184e-05
max,1.000000,0.117423,0.088530,8.400010e-03,3.879065e-03


In [15]:
fip_pop_pct_dict = census_df.POPESTIMATE2019.rank(pct=True).to_dict()

fip_s_dict = sir_df.SUSCEPTIBLE.to_dict()
fip_i_dict = sir_df.INFECTED.to_dict()
fip_r_dict = sir_df.REMOVED.to_dict()
fip_beta_dict = hyperparam_df.beta.dropna().to_dict()
fip_gamma_dict = hyperparam_df.gamma.dropna().to_dict()

def generate_data(fip, mode, dict1, dict2):
    try:
        return dict1[fip]
    except KeyError:
        # Generate default values based on infection rate percentile
        return dict2[fip]*(corona_quartiles[mode]['75%']-corona_quartiles[mode]['25%']) + corona_quartiles[mode]['25%']
    
matrix_df = census_df.copy()
matrix_df['SUSCEPTIBLE'] = [generate_data(i, 'SUSCEPTIBLE', fip_s_dict, fip_pop_pct_dict) for i in matrix_df.index]
matrix_df['INFECTED'] = [generate_data(i, 'INFECTED', fip_i_dict, fip_pop_pct_dict) for i in matrix_df.index]
matrix_df['REMOVED'] = [generate_data(i, 'REMOVED', fip_r_dict, fip_pop_pct_dict) for i in matrix_df.index]

inf_dict = matrix_df.INFECTED.rank(pct=True).to_dict()
rem_dict = matrix_df.REMOVED.rank(pct=True).to_dict()

matrix_df['beta'] = [generate_data(i, 'beta', fip_beta_dict, inf_dict) for i in matrix_df.index]
matrix_df['gamma'] = [generate_data(i, 'gamma', fip_gamma_dict, rem_dict) for i in matrix_df.index]

matrix_df['US'] = 1
matrix_df.drop(columns=['POPESTIMATE2019'], inplace=True)

display(matrix_df.describe())
matrix_df

,SUSCEPTIBLE,INFECTED,REMOVED,beta,gamma,US
count,3142.000000,3142.000000,3142.000000,3.142000e+03,3.142000e+03,3142.0
mean,0.998107,0.000748,0.001043,6.520613e-05,5.609916e-05,1.0
std,0.004491,0.003012,0.002605,2.170563e-04,1.303820e-04,0.0
min,0.880539,0.000000,0.000000,6.436166e-07,1.694066e-21,1.0
25%,0.998204,0.000073,0.000246,1.962307e-05,2.193552e-05,1.0
50%,0.999126,0.000202,0.000447,3.024309e-05,3.409521e-05,1.0
75%,0.999602,0.000627,0.000983,5.299188e-05,5.155486e-05,1.0
max,1.000000,0.117423,0.088530,8.400010e-03,3.879065e-03,1.0


,SUSCEPTIBLE,INFECTED,REMOVED,beta,gamma,US
FIPS,,,,,,
1001,0.999051,0.000412,0.000537,0.000032,0.000020,1
1003,0.999158,0.000291,0.000551,0.000023,0.000026,1
1005,0.998177,0.000689,0.001134,0.000067,0.000061,1
1007,0.998124,0.000447,0.001429,0.000032,0.000089,1
1009,0.999308,0.000242,0.000450,0.000022,0.000019,1
...,...,...,...,...,...,...
56037,0.999575,0.000047,0.000378,0.000013,0.000043,1
56039,0.995823,0.000256,0.003921,0.000110,0.000032,1
56041,0.999604,0.000049,0.000346,0.000028,0.000041,1


### Combine param matrix with county matrix and export

In [16]:
matrix_dict = dict(zip(matrix_df.index, matrix_df.to_numpy()))
matrix_dict

{1001: array([9.99051352e-01, 4.11677317e-04, 5.36970413e-04, 3.19625246e-05,
        2.01363905e-05, 1.00000000e+00]),
 1003: array([9.99157834e-01, 2.91174283e-04, 5.50991336e-04, 2.27179935e-05,
        2.57577251e-05, 1.00000000e+00]),
 1005: array([9.98177104e-01, 6.88649437e-04, 1.13424613e-03, 6.65501557e-05,
        6.07631856e-05, 1.00000000e+00]),
 1007: array([9.98124498e-01, 4.46548183e-04, 1.42895418e-03, 3.18962988e-05,
        8.93096365e-05, 1.00000000e+00]),
 1009: array([9.99308270e-01, 2.42105627e-04, 4.49624736e-04, 2.22341903e-05,
        1.94549165e-05, 1.00000000e+00]),
 1011: array([9.98415998e-01, 4.95000495e-04, 1.08900109e-03, 3.53571782e-05,
        8.66250866e-05, 1.00000000e+00]),
 1013: array([9.94138215e-01, 5.09049774e-03, 7.71287536e-04, 3.67279779e-04,
        4.49917729e-05, 1.00000000e+00]),
 1015: array([9.99075745e-01, 1.93653448e-04, 7.30601646e-04, 1.57186241e-05,
        2.31063774e-05, 1.00000000e+00]),
 1017: array([9.90918386e-01, 1.29307752

In [17]:
county_np = np.loadtxt('county_matrix.csv', delimiter=',', dtype=int)
county_np.shape

(1792, 2944)

In [18]:
matrix = np.zeros([county_np.shape[0], county_np.shape[1], 6])

for i in range(county_np.shape[0]):
    for j in range(county_np.shape[1]):
        try:
            # set parameter values based on the county
            matrix[i, j] = matrix_dict[county_np[i, j]]
            # Add noise to the matrix
            matrix[i, j, :5] = matrix[i, j, :5] * np.random.uniform(0.9, 1.1, 5)
        except KeyError:
            pass
        
matrix.shape

(1792, 2944, 6)

In [19]:
np.save('matrix', matrix)